# Burgers Equation
Equation:   $u_{t} + uu_{x}-\frac{0.01}{\pi}u_{xx} = 0$  
Boundary Conditions:  
$x \in [-1,1]$  $t \in [0,1]$  
$u(0,x)= -\sin(\pi x)$  
$u(t,-1)=u(t,1)=0$ 

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
pip install pyDOE    #required for latin hypercube sampling of collocation points

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=e5e49bc7a8f907a807e454cfeccca88471cb26355ef2f51602dea52ad776780d
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE


In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, InputLayer, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import glorot_normal
from pyDOE import lhs

## Data Prepocessing

In [ ]:
nu = 0.01/np.pi
N_u = 100                                                                       #boundary points
N_f = 10000                                                                     #collacation points
layers = [2, 20, 20, 20, 20, 20, 20, 20, 20, 1]
data = scipy.io.loadmat('burgers_shock.mat')                                    #contains x,t and exact usol
t = data['t'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = np.real(data['usol']).T
X, T = np.meshgrid(x,t)
X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))                  #2 columns containing x,t values
u_star = Exact.flatten()[:,None]                                                #1 column containing exact u values 
lb = X_star.min(0)                                                              #lower & upper bounds for x & t
ub = X_star.max(0) 
xx1 = np.hstack((X[0:1,:].T, T[0:1,:].T))
uu1 = Exact[0:1,:].T
xx2 = np.hstack((X[:,0:1], T[:,0:1]))
uu2 = Exact[:,0:1]
xx3 = np.hstack((X[:,-1:], T[:,-1:]))
uu3 = Exact[:,-1:]

X_u_train = np.vstack([xx1, xx2, xx3])
X_f_train = lb + (ub-lb)*lhs(2, N_f)                                            #Latin Hypercube Sampling method to generate collacation points
X_f_train = np.vstack((X_f_train, X_u_train))
u_train = np.vstack([uu1, uu2, uu3])
idx = np.random.choice(X_u_train.shape[0], N_u, replace=False)                  #Randomly choosing 100 training points
X_u_train = X_u_train[idx, :]
u_train = u_train[idx,:]

## Burger's Equation PINN (Formulation & Implementation)

In [ ]:
X_u = tf.convert_to_tensor(X_u_train[:,0:1])         #x boundary points 
T_u = tf.convert_to_tensor(X_u_train[:,1:2])         #t boundary points 
X_f = tf.convert_to_tensor(X_f_train[:,0:1])         #x collocation points
T_f = tf.convert_to_tensor(X_f_train[:,1:2])         #t collocation points
u_train = tf.convert_to_tensor(u_train)

def PINN(layers, lb, ub):
  model = Sequential()
  model.add(InputLayer(layers[0],))
  model.add(Lambda(lambda X: 2.0*(X - lb)/(ub - lb) - 1.0))
  for i in layers[1:-1]:
    model.add(Dense(units=i, activation="tanh", kernel_initializer="glorot_normal"))
  model.add(Dense(units=layers[-1], kernel_initializer="glorot_normal"))
  return model
  
def custom_loss(model, X_f, T_f, nu):
  def loss(u_pred, u_train):
    with tf.GradientTape(persistent=True) as tape:
      tape.watch(X_f)
      tape.watch(T_f)
      xf = tf.stack([X_f[:,0], T_f[:,0]], axis=1) 
      u = tf.cast(model(xf), dtype='float64')
      u_x = tape.gradient(u, X_f)
    u_xx = tape.gradient(u_x, X_f)
    u_t = tape.gradient(u, T_f)
    del tape
    u_train = tf.cast(u_train, dtype='float64')
    u_pred = tf.cast(u_pred, dtype='float64')
    f = u_t + u*u_x - nu*u_xx
    loss1 = tf.reduce_mean(tf.square(u_pred - u_train))
    loss2 = tf.reduce_mean(tf.square(f))
    return loss1 + loss2
  return loss

pinn_model = PINN(layers, lb, ub)
loss_fn = custom_loss(pinn_model, X_f, T_f, nu)
xu = tf.stack([X_u[:,0], T_u[:,0]], axis=1)
pinn_model.compile(optimizer=Adam(learning_rate=0.001), loss=loss_fn, metrics=['mse'])
pinn_model.fit(xu, u_train, batch_size=20, epochs=5000, verbose=2)

Streaming output truncated to the last 5000 lines.
Epoch 2501/5000
5/5 - 0s - loss: 2.3587e-04 - mse: 6.2687e-05
Epoch 2502/5000
5/5 - 0s - loss: 2.6901e-04 - mse: 6.0684e-05
Epoch 2503/5000
5/5 - 0s - loss: 2.3371e-04 - mse: 6.6524e-05
Epoch 2504/5000
5/5 - 0s - loss: 2.1886e-04 - mse: 6.3620e-05
Epoch 2505/5000
5/5 - 0s - loss: 1.9446e-04 - mse: 4.8398e-05
Epoch 2506/5000
5/5 - 0s - loss: 1.5886e-04 - mse: 4.5360e-05
Epoch 2507/5000
5/5 - 0s - loss: 2.2661e-04 - mse: 5.1192e-05
Epoch 2508/5000
5/5 - 0s - loss: 3.9452e-04 - mse: 4.6037e-05
Epoch 2509/5000
5/5 - 0s - loss: 8.8315e-04 - mse: 4.7950e-05
Epoch 2510/5000
5/5 - 0s - loss: 0.0017 - mse: 5.5749e-05
Epoch 2511/5000
5/5 - 0s - loss: 0.0012 - mse: 5.1613e-05
Epoch 2512/5000
5/5 - 0s - loss: 2.9919e-04 - mse: 5.0362e-05
Epoch 2513/5000
5/5 - 0s - loss: 4.1286e-04 - mse: 4.6998e-05
Epoch 2514/5000
5/5 - 0s - loss: 1.7310e-04 - mse: 4.8441e-05
Epoch 2515/5000
5/5 - 0s - loss: 2.5297e-04 - mse: 4.8140e-05
Epoch 2516/5000
5/5 - 0s - 

## Prediction

In [ ]:
u_pinn = pinn_model.predict(X_star)
table = np.hstack((u_pinn, u_star))
print('Predicted   -   Actual')
print(table[10000:10010])

Predicted   -   Actual
[[0.17373025 0.17630153]
 [0.18474826 0.18726152]
 [0.19578421 0.19821068]
 [0.20683342 0.20914833]
 [0.21789145 0.22007379]
 [0.22895393 0.23098638]
 [0.24001592 0.2418854 ]
 [0.25107393 0.25277013]
 [0.26212287 0.26363987]
 [0.2731601  0.2744939 ]]
